In [81]:
import requests
from bs4 import BeautifulSoup
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.platypus import BaseDocTemplate
from reportlab.lib.styles import getSampleStyleSheet
import sys
sys.path.append('/home/amandeep/Documents/Application/application/lib/python3.5/site-packages')

def Chapter(url,textToPdf,chapterTitle,nextChapterUrl,divName):
    h2 = PS(name = 'Heading2',
 fontSize = 12,
 leading = 14,
 )
    
    styles = getSampleStyleSheet()
    r=requests.get(url)
    #print(requests.get(url))
    #print(r.text)
    #print(soup.select("kmk=vfkmv div"))
    headerCheck=0
    nexturlCheck=0
    s = ""
    
    soup = BeautifulSoup(r.text, 'lxml')
    for divChapter in soup.findAll("div",{"itemprop":"articleBody"}):
        for parChapter in divChapter.findAll("p"):
            if "Previous Chapter" not in parChapter.text and "Next Chapter" not in parChapter.text: #Can change this with regex
                if headerCheck==0:
                    chapterTitle.append('<a href="#%s">%s</a>'%(str(divName),parChapter.text,))
                    textToPdf.append(Paragraph('<a name="%s"/><strong>%s</strong>' % (str(divName),parChapter.text,), styles["BodyText"]))
                    #s+=str(TitleChapter)
                    #s+='\n'
                    #print(TitleChapter)
                    #print('_________________________________________________')
                    textToPdf.append(Paragraph('_________________________________________________', styles["Normal"]))
                    textToPdf.append(Paragraph('<br/>', styles["BodyText"]))
                    textToPdf.append(Paragraph('<br/>', styles["BodyText"]))
                    headerCheck=1

                else:
                    textToPdf.append(Paragraph(parChapter.text, styles["BodyText"]))

                    textToPdf.append(Paragraph('<br/>', styles["BodyText"]))
                    s+=str(parChapter)
                    #print(parChapter)
            else:
                for urlChapter in parChapter.find_all('a', href=True):
                    if urlChapter.text=="Next Chapter":
                        print(urlChapter['href'])
                        nexturlCheck=1
                        nextUrl=urlChapter['href']
                        nextChapterUrl+=nextUrl
        #print d.a.get("data-name")
        #print d.select("a.results")
    return nexturlCheck

#Story.append(Paragraph('_________________________________________________', styles["Normal"]))
#doc.build(Story)

In [39]:
'''
Function to loop through all pages
'''
def ChaptersAdd(url,ChapterContent,firstChapter,lastChapter,chapterTitle):
    currentChapter=firstChapter
    nexturlCheck=0
    while currentChapter <= lastChapter or lastChapter==0:
        urlChapter = '%s-%s/'%(url,currentChapter)
        nextUrl=""
        nexturlCheck=Chapter(urlChapter,ChapterContent,chapterTitle,nextUrl,currentChapter)
        ChapterContent.append(PageBreak())
        if(nexturlCheck==0):
            break
        elif chapterTitle is urlChapter:
            break
        currentChapter+=1

In [37]:
from reportlab.pdfgen import canvas

def hello(c):()
    c.drawString(100,100,"Hello World")
c = canvas.Canvas("hello.pdf")
hello(c)

from reportlab.lib.units import inch
c.drawString( 20,500, s)

#textobject=canvas.beginText(canvas,0,0)
textobject = c.beginText(400,500)
textobject.setTextOrigin(inch, 2.5*inch)
textobject.setFont("Helvetica-Oblique", 14)
textobject.textLines('With many apologies to the Beach Boysand anyone else who finds this objectionable')
c.drawText(textobject)
c.showPage()
c.save()

In [57]:
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

very_long_text = "Your very long text here ..."


styles = getSampleStyleSheet()
doc = SimpleDocTemplate("my_doc.pdf", pagesize=A4)
Story=[]
Story.append(Paragraph(very_long_text, styles["BodyText"]))
doc.build(Story)

In [85]:
from reportlab.pdfgen import canvas
from reportlab.platypus import (SimpleDocTemplate, Paragraph, PageBreak)
from reportlab.platypus.tableofcontents import TableOfContents
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet

class FooterCanvas(canvas.Canvas):
    '''
     Juan Fco. Roco Author
     http://stackoverflow.com/questions/28281108/reportlab-how-to-add-a-footer-to-a-pdf-file
     Edited : To add title/chapter name
    '''
    def __init__(self, *args, **kwargs):
        canvas.Canvas.__init__(self, *args, **kwargs)
        self.pages = []
        #self.chapterTitle=[]

    def showPage(self):
        self.pages.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        page_count = len(self.pages)
        for page in self.pages:
            self.__dict__.update(page)
            self.draw_canvas(page_count)
            canvas.Canvas.showPage(self)
        canvas.Canvas.save(self)

    def draw_canvas(self, page_count):
        page = "Page %s of %s" % (self._pageNumber, page_count)
        x = 128
        self.saveState()
        self.setStrokeColorRGB(0, 0, 0)
        self.setLineWidth(0.5)
        self.line(66, 78, A4[0] - 66, 78)
        self.setFont('Times-Roman', 10)
        self.drawString(A4[0]-x, 65, page)
        self.drawString(x, 65, self.title)
        self.restoreState()


if __name__ == '__main__':
    
    h2 = PS(name = 'Heading2',
 fontSize = 12,
 leading = 14,
 )
    toc.levelStyles = [h2]
    toc = TableOfContents()
    story.append(toc)
    story.append(PageBreak())

    #Story.append(PageBreak())
    styles = getSampleStyleSheet()
    
    url = 'http://www.wuxiaworld.com/rmji-index/rmji-chapter-'
    doc = SimpleDocTemplate("my_doc.pdf", pagesize=A4)
    Story=[]
    chapterTitle=[]
    ChaptersAdd(url,Story,334,0,chapterTitle)
    chapterTitleString=""
    '''for title in chapterTitle:
        chapterTitleString+="%s<br/><br/>"%(title,)
    Story.insert(0,Paragraph(chapterTitleString, styles["BodyText"]))'''
    i=0
    for title in chapterTitle:
        Story.insert(i,Paragraph(title, styles["BodyText"]))
        i+=1
    Story.insert(i,PageBreak())
    #doc.build(Story)
    

    footer=FooterCanvas
    footer.title='A Record of a Mortal’s Journey to Immortality'
    doc.multiBuild(Story,canvasmaker=footer)

http://www.wuxiaworld.com/rmji-index/rmji-chapter-335
http://www.wuxiaworld.com/rmji-index/rmji-chapter-335
http://www.wuxiaworld.com/rmji-index/rmji-chapter-336
http://www.wuxiaworld.com/rmji-index/rmji-chapter-336
http://www.wuxiaworld.com/rmji-index/rmji-chapter-337
http://www.wuxiaworld.com/rmji-index/rmji-chapter-337
http://www.wuxiaworld.com/rmji-index/rmji-chapter-337


In [44]:
from reportlab.lib.styles import ParagraphStyle as PS
from reportlab.platypus import PageBreak
from reportlab.platypus.paragraph import Paragraph
from reportlab.platypus.doctemplate import PageTemplate, BaseDocTemplate
from reportlab.platypus.tableofcontents import TableOfContents
from reportlab.platypus.frames import Frame
from reportlab.lib.units import cm
class MyDocTemplate(BaseDocTemplate):
     def __init__(self, filename, **kw):
         self.allowSplitting = 0
         BaseDocTemplate.__init__(self, filename, **kw)
         template = PageTemplate('normal', [Frame(2.5*cm, 2.5*cm, 15*cm, 25*cm, id='F1')])
         self.addPageTemplates(template)
            
     def afterFlowable(self, flowable):
         "Registers TOC entries."
         if flowable.__class__.__name__ == 'Paragraph':
             text = flowable.getPlainText()
             style = flowable.style.name
             if style == 'Heading1':
                 self.notify('TOCEntry', (0, text, self.page))
             if style == 'Heading2':
                 self.notify('TOCEntry', (1, text, self.page))
h1 = PS(name = 'Heading1',
 fontSize = 14,
 leading = 16)
h2 = PS(name = 'Heading2',
 fontSize = 12,
 leading = 14,
 )
# Build story.
story = []
toc = TableOfContents()
# For conciseness we use the same styles for headings and TOC entries
toc.levelStyles = [h1, h2]
story.append(toc)
story.append(PageBreak())
story.append(Paragraph('First heading', h1))
story.append(Paragraph('Text in first heading', PS('body')))
story.append(Paragraph('First sub heading', h2))
story.append(Paragraph('Text in first sub heading', PS('body')))
story.append(PageBreak())
story.append(Paragraph('Second sub heading', h2))
story.append(Paragraph('Text in second sub heading', PS('body')))
story.append(Paragraph('Last heading', h1))
doc = MyDocTemplate('mintoc.pdf')
doc.multiBuild(story)

2